In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import scipy.stats as sps

In [2]:
Y_train = pd.read_csv('y_train.csv')
X_test= pd.read_csv('X_test_scaled.csv')
Y_test= pd.read_csv('y_test.csv')
X_train = pd.read_csv('X_train_scaled.csv')
Y_oot = pd.read_csv('y_oot.csv')
X_oot = pd.read_csv('X_oot_scaled.csv')

In [4]:
regr = AdaBoostClassifier(DecisionTreeClassifier(criterion='gini'),
                        n_estimators=300)
# loss function ‘linear’, ‘square’, ‘exponential’

In [5]:
regr.fit(X_train, Y_train)

C:\Users\heyup\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=300, random_state=None)

In [6]:
pred_train = regr.predict_proba(X_train)[:,1]
pred_train

array([2.22044605e-16, 2.22044605e-16, 2.22044605e-16, ...,
       2.22044605e-16, 2.22044605e-16, 2.22044605e-16])

In [7]:
pred_test= regr.predict_proba(X_test)[:,1]
pred_test

array([2.22044605e-16, 2.22044605e-16, 2.22044605e-16, ...,
       2.22044605e-16, 2.22044605e-16, 2.22044605e-16])

In [8]:
pred_oot = regr.predict_proba(X_oot)[:,1]
pred_oot

array([1.00000000e+00, 1.00000000e+00, 2.22044605e-16, ...,
       2.22044605e-16, 2.22044605e-16, 2.22044605e-16])

In [9]:
#test
newDF = X_test
newDF['prob'] = pred_test
newDF['org'] = Y_test
newDF.sort_values(by=['prob'], ascending = False, inplace = True)

In [10]:
#train
newDF1 = X_train
newDF1['prob'] = pred_train
newDF1['org'] = Y_train
newDF1.sort_values(by=['prob'], ascending = False, inplace = True)

In [12]:
#oot
newDF2 = X_oot
newDF2['prob'] = pred_oot
newDF2['org'] = Y_oot
newDF2.sort_values(by=['prob'], ascending = False, inplace = True)

## FDR 

In [31]:
# test

fdr = []
records_cum = []
goods_cum = []
bads_cum = []
perc_goods = []
perc_bads = []

total_bads = sum(newDF['org']==1)
total_goods = sum(newDF['org']==0)
total_records = len(newDF)
fraud_rate = total_bads / total_records

for i in np.arange(0.01, 0.21, 0.01):
    topRows = int(round(len(newDF)*i))
    records_cum.append(topRows)
    bads_cum.append(newDF.iloc[0:topRows, 21].sum())
    goods_cum.append(topRows - newDF.iloc[0:topRows, 21].sum())
    perc_goods.append((topRows - newDF.iloc[0:topRows, 21].sum())/ total_goods)
    fdr.append(newDF.iloc[0:topRows, 21].sum()/sum(newDF['org']==1))
    

print(total_bads, total_goods, total_records, fraud_rate)

253 24938 25191 0.010043269421618832


In [29]:
test_stats = pd.DataFrame({'fdr':fdr,
                          'records_cum':records_cum,
                          'goods_cum':goods_cum,
                          'bads_cum':bads_cum,
                          'perc_goods': perc_goods})
test_stats
#test_stats.to_csv('Test stats.csv')

,fdr,records_cum,goods_cum,bads_cum,perc_goods
0,0.075099,252,233,19,0.009343
1,0.177866,504,459,45,0.018406
2,0.260870,756,690,66,0.027669
3,0.347826,1008,920,88,0.036891
4,0.359684,1260,1169,91,0.046876
5,0.367589,1511,1418,93,0.056861
6,0.379447,1763,1667,96,0.066846
7,0.379447,2015,1919,96,0.076951
8,0.391304,2267,2168,99,0.086936
9,0.395257,2519,2419,100,0.097001


In [121]:
# train# test
topRows = int(round(len(newDF1)*0.03))

newDF1.iloc[0:topRows, 21].sum()/sum(newDF1['org']==1)


0.009569377990430622

In [122]:
# 00t
topRows = int(round(len(newDF2)*0.03))

newDF2.iloc[0:topRows, 21].sum()/sum(newDF2['org']==1)

0.027932960893854747